In [2]:
# ============================================
# 📌 設定 & 核心模組載入
# ============================================
import sys
from pathlib import Path
import pandas as pd
import numpy as np
from datetime import datetime
import torch

# --- 1. 定義根目錄 ---
root_path = Path().resolve()
while root_path.name != "YOLO11_Experiment" and root_path != root_path.parent:
    root_path = root_path.parent

sys.path.append(str(root_path / "utils"))

# --- 2. 實驗參數 ---
EXP_ID = "EXP_00_TEST" 
detector_path = "weights/combined_freeze10.pt"
tracker_type = 'botsort'

# --- 3. 投票參數 ---
VOTE_METHOD = 'weighted'  # 投票修正方式: 'majority', 'weighted'
WINDOW_SIZE = 5           # 窗口大小: 3, 5, 10, 'all'
IS_REALTIME = True        # Real-time 模擬
HALF_PRECISION = True     # 啟用 FP16 半精度加速

# --- 4. 匯入所有函式 ---
from source_utils import detect_source_type, detect_validation_yaml
from yolo_detector import load_model, run_validation, run_predict_on_validation_set
from evaluation_metrics import compute_class_stability, compute_class_stability_with_id, compute_mot_metrics
from tracking_launcher import run_tracker
from voting_mechanism import run_class_voting
from output_formatter import get_detailed_name
from visualization import generate_video_from_csv

# ============================================
# ⚙️ 輔助函式定義
# ============================================
def ensure_dataframe(metrics):
    """
    確保指標是 DataFrame 格式。
    如果是字典 (dict)，則轉換為單列 DataFrame。
    """
    if metrics is None:
        return None
    if isinstance(metrics, dict):
        # 將字典轉為 DataFrame (index=[0])
        return pd.DataFrame([metrics])
    return metrics
    
def log_experiment_result(log_path, data_dict):
    """
    將單次實驗的結果 (mAP, MOTA, IDSW 等) 寫入總表 CSV。
    """
    df_new = pd.DataFrame([data_dict])
    if not log_path.exists():
        df_new.to_csv(log_path, index=False)
    else:
        df_new.to_csv(log_path, mode='a', header=False, index=False)
    print(f"📝 數據已寫入總表: {log_path}")

def print_evaluation_summary(label, stability_metrics, mot_metrics=None, map_metrics=None, 
                             avg_latency_ms=None, system_latencies=None):
    """
    統一列印評估摘要。此函式會根據 system_latencies 顯示總體 Pipeline FPS。
    
    Args:
        label (str): 階段標籤。
        stability_metrics (dict): 穩定性指標。
        mot_metrics (pd.DataFrame, optional): MOT 指標。
        map_metrics (Box, optional): mAP 指標。
        avg_latency_ms (float): 該階段的基礎延遲 (Det/Track)。
        system_latencies (dict, optional): 僅在 Stage 3 傳入，包含 {'track': float, 'vote': float}。
    """
    print(f"\n=============================================")
    print(f"✅ 評估結果：{label}")
    print(f"=============================================")

    # 1. 偵測基礎指標 (mAP)
    if map_metrics:
        print("--- 偵測準確度 (mAP) ---")
        print(f"  > mAP@0.5:        {map_metrics.map50:.4f}")
        print(f"  > mAP@0.5:0.95:   {map_metrics.map:.4f}")
        
    # 2. 穩定性指標
    if stability_metrics:
        if 'class_instability_rate' in stability_metrics:
            print("\n--- 跨幀類別穩定性 (IoU 匹配) ---")
            print(f"  > 類別不穩定率:       {stability_metrics['class_instability_rate']:.4f}")
            print(f"  > 總 IoU 比較對數: {stability_metrics['total_matched_pairs']}")
            
        if 'class_instability_rate_id' in stability_metrics:
            print("\n--- 跨幀類別穩定性 (基於 ID) ---")
            print(f"  > 類別不穩定率: {stability_metrics['class_instability_rate_id']:.4f}")
            print(f"  > 總 ID 比較對數:  {stability_metrics['total_comparisons_id']}")
            if 'total_class_changes_id' in stability_metrics:
                 print(f"  > 總類別變換次數:  {stability_metrics['total_class_changes_id']}")

    # 3. MOT 指標
    if mot_metrics is not None:
        print("\n--- 多目標追蹤 (MOT) 指標 ---")
        try:
            if not mot_metrics.empty:
                print(f"  > MOTA:          {mot_metrics['MOTA'].iloc[0]:.4f}")
                print(f"  > IDF1:          {mot_metrics['IDF1'].iloc[0]:.4f}")
                print(f"  > IDSW (ID 轉換): {int(mot_metrics['IDSW'].iloc[0])}")
                print(f"  > FP (誤檢):     {int(mot_metrics['FP'].iloc[0])}")
                print(f"  > FN (漏檢):     {int(mot_metrics['FN'].iloc[0])}")
                if 'Cls_Acc' in mot_metrics:
                     print(f"  > 分類準確率:    {mot_metrics['Cls_Acc'].iloc[0]:.4f}")
            else:
                print("  > MOT 指標 DataFrame 為空。")
        except Exception as e:
            print(f"⚠️ MOT 指標解析失敗: {e}")
            
    # 4. 運算效能 (Performance)
    if avg_latency_ms is not None:
        base_latency = avg_latency_ms
        base_fps = 1000.0 / base_latency if base_latency > 0 else 0.0
        
        if system_latencies and 'track' in system_latencies and 'vote' in system_latencies:
            # === Stage 3: Tracking + Voting Pipeline ===
            latency_track = system_latencies['track']
            latency_voting = system_latencies['vote']
            total_latency = latency_track + latency_voting
            total_fps = 1000.0 / total_latency if total_latency > 0 else 0.0

            print(f"\n--- 總體 Pipeline 效能 ---")
            print(f"  > 基礎追蹤延遲 (T_Track): {latency_track:.4f} ms/frame")
            print(f"  > 投票修正延遲 (T_Vote):  {latency_voting:.4f} ms/frame")
            print(f"  > 總體 Pipeline 延遲:    {total_latency:.4f} ms/frame")
            print(f"  > 總體 Pipeline FPS:     {total_fps:.2f} FPS")
            
        else:
            # === Stage 1/2: 單一模組效能 (Det / Raw Track) ===
            print(f"\n--- 單一模組效能 ---")
            print(f"  > 平均處理延遲:  {base_latency:.4f} ms/frame")
            print(f"  > 平均處理 FPS:    {base_fps:.2f} FPS")
            
# ============================================
# 🚀 主程式
# ============================================
if __name__ == "__main__":
    
    # ============================================
    # 📂 資料載入與模型初始化
    # ============================================
    raw_path = root_path / "data" / "IMG_0004"
    source_path = detect_source_type(raw_path)
    
    # 取得序列名稱 (例如: IMG_0004) - 用於檔名生成
    if source_path.is_dir():
        # 如果 source 指向 images 資料夾，取上一層
        sequence_name = source_path.parent.name if source_path.name == "images" else source_path.name
    else:
        sequence_name = source_path.stem
    
    yaml_path = detect_validation_yaml(raw_path)
    gt_path = raw_path / 'gt' / 'gt.txt' # 真值路徑
    
    # ---------------------------------------------------------
    # 🔧 路徑設定
    # 結構: YOLO_experiments/results/00_test/[實驗資料夾]
    # ---------------------------------------------------------
    
    # 1. 定義「測試總目錄」
    TEST_ROOT_DIR = root_path / "results" / "test"
    
    # 2. 動態生成「本次實驗專屬資料夾」名稱
    folder_name = get_detailed_name(
        seq=sequence_name,    # IMG_0004
        stage="Exp",          # 通用標籤
        tracker=tracker_type, # botsort
        method=VOTE_METHOD,   # weighted
        win=WINDOW_SIZE,      # 5
        realtime=IS_REALTIME, # True
        ext=""                # 資料夾不需副檔名
    )
    
    # 3. 組合最終路徑並建立
    output_dir = TEST_ROOT_DIR / folder_name
    output_dir.mkdir(parents=True, exist_ok=True)
    
    # --- 載入模型 ---
    model = load_model(root_path / detector_path)
    
    print("\n=============================================")
    print(f"✅ 實驗 ID: {EXP_ID}")
    print(f"📂 實驗目錄: {folder_name}")
    print(f"📍 完整輸出路徑: {output_dir}")
    print("=============================================")
    print(f"   - 來源: {source_path}")
    print(f"   - 設定: {yaml_path}")
    print(f"   - 真值: {gt_path}")
    
    print(f"\n=== 開始實驗: {sequence_name} | {tracker_type} | Vote: {VOTE_METHOD} (w={WINDOW_SIZE}) ===")
    
    # ============================================
    # 📊 階段 I: 僅偵測 (DETECTION ONLY)
    # ============================================
    print("\n--- 開始評估階段 I: 僅偵測基準線 ---")
    
    # 1. 執行 Validation 取得 mAP
    metrics = run_validation(model, yaml_path, output_dir=output_dir)

    del model
    torch.cuda.empty_cache()
    model = load_model(root_path / detector_path)

    # 2. 執行 Predict 取得未追蹤的 CSV (用於 IoU 穩定性計算)
    csv_path_detection, latency_detection = run_predict_on_validation_set(model, source_path=source_path, output_dir=output_dir, half=HALF_PRECISION)
    
    # 3. 計算類別穩定性 (IoU 匹配)
    stability_metrics_detection = compute_class_stability(csv_path_detection)
    
    # 4. 輸出結果 I
    print_evaluation_summary("1. 基準線 (僅偵測器)", 
                             stability_metrics=stability_metrics_detection, 
                             map_metrics=metrics.box, 
                             avg_latency_ms=latency_detection)
    
    # ============================================
    # 📦 階段 II: 追蹤 (RAW Tracking)
    # ============================================
    print("\n--- 開始評估階段 II: 原始追蹤結果 (未修正) ---")
    
    # A. 執行追蹤 (run_tracker 回傳 DF 與 Latency)
    df_raw_tracking, latency_tracking = run_tracker(tracker_type=tracker_type, source_path=source_path, output_dir=output_dir, model=model, half=HALF_PRECISION)
    
    if df_raw_tracking.empty:
        print("❌ 追蹤器未偵測到任何目標，無法繼續追蹤評估。")
        sys.exit()
    
    # B. 儲存 CSV
    raw_tracking_csv_name = get_detailed_name(sequence_name, "02-RawTracking", tracker_type, ext=".csv")
    raw_tracking_csv_path = output_dir / raw_tracking_csv_name 
    df_raw_tracking.to_csv(raw_tracking_csv_path, index=False)
    print(f"✅ 原始追蹤 CSV 儲存在: {raw_tracking_csv_path}")
    
    # C. 評估 Raw
    stability_metrics_raw = compute_class_stability_with_id(raw_tracking_csv_path)
    mot_metrics_raw = compute_mot_metrics(raw_tracking_csv_path, gt_path)
    # 強制轉型為 DataFrame，避免 dict 報錯
    mot_metrics_raw = ensure_dataframe(mot_metrics_raw)
    
    print_evaluation_summary("2. 原始追蹤 (RAW Tracking)", 
                             stability_metrics=stability_metrics_raw, 
                             mot_metrics=mot_metrics_raw, 
                             avg_latency_ms=latency_tracking)
    
    # ============================================
    # 📊 階段 III: 追蹤 + 投票修正 (VOTED)
    # ============================================
    print(f"\n--- 開始評估階段 III: 追蹤 + 投票修正 ---")
    
    # 1. 執行投票
    df_corrected_tracking, latency_voting = run_class_voting(
        input_df=df_raw_tracking, method=VOTE_METHOD, window_size=WINDOW_SIZE, is_realtime=IS_REALTIME
    )
    
    # 2. 儲存 CSV
    corrected_tracking_csv_name = get_detailed_name(sequence_name, "03-Voted", tracker_type, VOTE_METHOD, WINDOW_SIZE, IS_REALTIME, ext=".csv")
    corrected_tracking_csv_path = output_dir / corrected_tracking_csv_name
    df_corrected_tracking.to_csv(corrected_tracking_csv_path, index=False)
    print(f"✅ 修正後 CSV 儲存在: {corrected_tracking_csv_path}")
    
    # 3. 評估 Voted
    stability_metrics_corrected = compute_class_stability_with_id(corrected_tracking_csv_path)
    mot_metrics_corrected = compute_mot_metrics(corrected_tracking_csv_path, gt_path)
    # 強制轉型為 DataFrame
    mot_metrics_corrected = ensure_dataframe(mot_metrics_corrected)
    
    # 4. Output III (總效能表)
    system_latencies = {
        'det': latency_detection,
        'track': latency_tracking,
        'vote': latency_voting
    }
    
    print_evaluation_summary("3. 追蹤 + 投票修正 (最終效能)", 
                             stability_metrics=stability_metrics_corrected, 
                             mot_metrics=mot_metrics_corrected,
                             avg_latency_ms=latency_tracking,
                             system_latencies=system_latencies)
    
    # ============================================
    # 🎬 視覺化
    # ============================================
    print("\n=============================================")
    print("🎬 正在生成比較影片 (Visualization)...")
    print("=============================================")
    
    # 1. 偵測階段影片
    vid_det_path = output_dir / f"{sequence_name}_01_Detection.mp4"
    generate_video_from_csv(source_path, csv_path_detection, vid_det_path, mode='detection')
    
    # 2. 原始追蹤影片
    vid_raw_path = output_dir / f"{sequence_name}_02_RawTracking.mp4"
    generate_video_from_csv(source_path, raw_tracking_csv_path, vid_raw_path, mode='raw')
    
    # 3. 修正後影片
    vid_voted_path = output_dir / f"{sequence_name}_03_VotedTracking.mp4"
    generate_video_from_csv(source_path, corrected_tracking_csv_path, vid_voted_path, mode='voted')
    
    # ============================================
    # 📝 實驗紀錄 (最終步驟)
    # ============================================
    log_file = root_path / "results" / "experiments_log.csv"
    
    total_latency_ms = latency_detection + latency_tracking + latency_voting
    system_fps = 1000.0 / total_latency_ms if total_latency_ms > 0 else 0
    
    fps_detection = 1000.0 / latency_detection if latency_detection > 0 else 0.0
    fps_tracking = 1000.0 / latency_tracking if latency_tracking > 0 else 0.0
    fps_voting = 1000.0 / latency_voting if latency_voting > 0 else 0.0

    # 整理最終要記錄的數據
    record = {
         "exp_id": EXP_ID,
         "date": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
         "task": "baseline_test",
         "detector": detector_path.split('/')[-1],
         "tracker": tracker_type,
         "voting_method": VOTE_METHOD,
         "window_size": WINDOW_SIZE,
         "realtime_mode": "RT" if IS_REALTIME else "Off",
         "dataset": sequence_name,
         # 準確度
         "mAP@0.5": metrics.box.map50,
         "mAP": metrics.box.map,
         "MOTA_RAW": mot_metrics_raw['MOTA'].iloc[0] if mot_metrics_raw is not None else 0,
         "IDF1_RAW": mot_metrics_raw['IDF1'].iloc[0] if mot_metrics_raw is not None else 0,
         "MOTA_VOTE": mot_metrics_corrected['MOTA'].iloc[0] if mot_metrics_corrected is not None else 0,
         "IDF1_VOTE": mot_metrics_corrected['IDF1'].iloc[0] if mot_metrics_corrected is not None else 0,
         "Cls_Acc_VOTE": mot_metrics_corrected['Cls_Acc'].iloc[0] if (mot_metrics_corrected is not None and 'Cls_Acc' in mot_metrics_corrected) else 0,
         # 穩定性
         "Stability_RAW_ID": stability_metrics_raw['class_instability_rate_id'],
         "Stability_VOTE_ID": stability_metrics_corrected['class_instability_rate_id'],
         # 效能 (Latency)
         "Latency_Detection": latency_detection,
         "Latency_Tracking": latency_tracking,
         "Latency_Voting": latency_voting,
         "Total_Latency": total_latency_ms,
         # 效能 (FPS)
         "FPS_Detection": fps_detection,
         "FPS_Tracking": fps_tracking,
         "FPS_Voting": fps_voting,
         "System_FPS": system_fps,
         # 路徑與備註
         "Video_Path": str(vid_voted_path),
         "notes": "系統總效能與視覺化驗證完成"
    }
    
    log_experiment_result(log_file, record)
    
    print(f"\n🎉 全部實驗完成！")
    print(f"   總表: {log_file}")
    print(f"   影片: {output_dir}")


✅ 實驗 ID: EXP_00_TEST
📂 實驗目錄: IMG_0004_Exp_botsort_weighted_w5_RT_20251205-125419
📍 完整輸出路徑: D:\experiment\YOLO11_Experiment\results\00_test\IMG_0004_Exp_botsort_weighted_w5_RT_20251205-125419
   - 來源: D:\experiment\YOLO11_Experiment\data\IMG_0004\images
   - 設定: D:\experiment\YOLO11_Experiment\data\IMG_0004\data.yaml
   - 真值: D:\experiment\YOLO11_Experiment\data\IMG_0004\gt\gt.txt

=== 開始實驗: IMG_0004 | botsort | Vote: weighted (w=5) ===

--- 開始評估階段 I: 僅偵測基準線 ---
Ultralytics 8.3.6  Python-3.12.3 torch-2.4.1 CUDA:0 (NVIDIA GeForce RTX 4070, 12282MiB)
YOLO11m summary (fused): 303 layers, 20,138,743 parameters, 0 gradients, 68.3 GFLOPs


val: Scanning D:\experiment\YOLO11_Experiment\data\IMG_0004\labels.cache... 301 images, 17 backgrounds, 0 corrupt: 100%
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:03


                   all        301        284          1      0.518      0.759        0.7
   Pycnonotus taivanus        284        284          1      0.518      0.759        0.7
Speed: 0.3ms preprocess, 8.9ms inference, 0.0ms loss, 0.4ms postprocess per image
Saving D:\experiment\YOLO11_Experiment\results\00_test\IMG_0004_Exp_botsort_weighted_w5_RT_20251205-125419\val\predictions.json...
Results saved to D:\experiment\YOLO11_Experiment\results\00_test\IMG_0004_Exp_botsort_weighted_w5_RT_20251205-125419\val

✅ 正在儲存驗證集偵測結果至標準化 CSV: D:\experiment\YOLO11_Experiment\results\00_test\IMG_0004_Exp_botsort_weighted_w5_RT_20251205-125419\IMG_0004_predictions.csv
   ⏱️ 偵測總運算耗時: 7172.47 ms

✅ 評估結果：1. 基準線 (僅偵測器)
--- 偵測準確度 (mAP) ---
  > mAP@0.5:        0.7588
  > mAP@0.5:0.95:   0.7001

--- 跨幀類別穩定性 (IoU 匹配) ---
  > 不穩定率:       0.0588
  > 總 IoU 比較對數: 153

--- 單一模組效能 ---
  > 平均處理延遲:  23.8288 ms/frame
  > 平均處理 FPS:    41.97 FPS

--- 開始評估階段 II: 原始追蹤結果 (未修正) ---

✅ 正在執行 botsort 追蹤，來源: D:\experiment\YOLO1

Rendering: 100%|█████████████████████████████████████████████████████████████████████| 301/301 [00:08<00:00, 34.82it/s]


✅ 影片已儲存。

🎬 正在生成影片 (raw): IMG_0004_02_RawTracking.mp4


Rendering: 100%|█████████████████████████████████████████████████████████████████████| 301/301 [00:06<00:00, 47.04it/s]


✅ 影片已儲存。

🎬 正在生成影片 (voted): IMG_0004_03_VotedTracking.mp4


Rendering: 100%|█████████████████████████████████████████████████████████████████████| 301/301 [00:06<00:00, 49.63it/s]

✅ 影片已儲存。

📝 數據已寫入總表: D:\experiment\YOLO11_Experiment\results\experiments_log.csv

🎉 全部實驗完成！
   總表: D:\experiment\YOLO11_Experiment\results\experiments_log.csv
   影片: D:\experiment\YOLO11_Experiment\results\00_test\IMG_0004_Exp_botsort_weighted_w5_RT_20251205-125419
